In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itkwidgets import view
import cv2
import multiprocessing as mp
import SharedArray as sa
from bioblue.nb import *
from bioblue.plot import cm
from bioblue.registration.utilities import display_images_with_alpha
from bioblue.registration import callbacks as regcbs
from ipywidgets import interact, fixed
import pickle

In [2]:
shared_mem = "image_sdffj"

def readimage(i, scanfile, shape):
    scan = cv2.imread(scanfile, cv2.IMREAD_UNCHANGED)
    scan = cv2.resize(scan, tuple(reversed(shape)), cv2.INTER_CUBIC)
    image = sa.attach(f"shm://{shared_mem}")
    image[:,:,i] = scan

In [8]:
scanspath = Path("~/projects/masks").expanduser()
images = {}
for scandir in scanspath.iterdir():
    if not scandir.is_dir():
        continue
    if "s4" not in scandir.name:
        continue
    print(scandir)
    suffix = next(scandir.glob("*")).suffix
    scanfiles = [str(x) for x in sorted((scandir).glob(f"*{suffix}"))]# [250:-250:4]
    scanfirst = cv2.imread(scanfiles[0], cv2.IMREAD_UNCHANGED)
    shape = tuple(int(x) for x in scanfirst.shape)
    print(shape)
    args = [(i,x,shape) for i,x in enumerate(scanfiles)]
    image = np.zeros(shape+(len(scanfiles),), dtype=scanfirst.dtype) # sa.create(f"shm://{shared_mem}", shape+(len(scanfiles),))
    for i, scanfile in enumerate(tqdm(scanfiles)):
        scan = cv2.imread(str(scanfile), cv2.IMREAD_UNCHANGED)
        image[:,:,i] = scan
        # scan = cv2.resize(scan, shape, cv2.INTER_CUBIC)
    images[scandir.name] = image

with open("tmp/masks.pickle", 'wb') as picklefile:
    pickle.dump(images, picklefile, protocol=pickle.HIGHEST_PROTOCOL)

/home/vjoosdeterbe/projects/masks/s4-1h-bin
(511, 2201)


  0%|          | 0/2000 [00:00<?, ?it/s]

/home/vjoosdeterbe/projects/masks/s4-2h
(526, 2052)


  0%|          | 0/1951 [00:00<?, ?it/s]

/home/vjoosdeterbe/projects/masks/s4-13mv2
(509, 2194)


...

  0%|          | 0/1951 [00:00<?, ?it/s]

/home/vjoosdeterbe/projects/masks/s4-13m
(546, 2563)


  0%|          | 0/1901 [00:00<?, ?it/s]

/home/vjoosdeterbe/projects/masks/s4-1h
(511, 2201)


  0%|          | 0/2000 [00:00<?, ?it/s]

In [2]:
import pickle
with open("tmp/masks.pickle", 'rb') as picklefile:
    images = pickle.load(picklefile)

In [3]:
sitk_s4_2h = sitk.PermuteAxes(sitk.GetImageFromArray(images["s4-2h"].astype(np.float32)), [1,2,0])
sitk_s4_1h = sitk.PermuteAxes(sitk.GetImageFromArray(images["s4-1h"].astype(np.float32)), [1,2,0])

In [4]:
del images

In [5]:
initial_transform = sitk.CenteredTransformInitializer(sitk_s4_2h, sitk_s4_1h, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY)
# print(initial_transform)
moving_resampled = sitk.Resample(sitk_s4_1h, sitk_s4_2h, initial_transform, sitk.sitkLinear, 0.0, sitk_s4_1h.GetPixelID())

In [6]:
print(initial_transform)

itk::simple::Transform
 Euler3DTransform (0x55a292d1fb60)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 1
   Modified Time: 745
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1 0 0 
     0 1 0 
     0 0 1 
   Offset: [24.5, 74.5, -7.5]
   Center: [975, 1025.5, 262.5]
   Translation: [24.5, 74.5, -7.5]
   Inverse: 
     1 0 0 
     0 1 0 
     0 0 1 
   Singular: 0
   Euler's angles: AngleX=0 AngleY=0 AngleZ=0
   m_ComputeZYX = 0



In [7]:
interact(display_images_with_alpha, image_z=(0, sitk_s4_2h.GetSize()[2]-1), alpha=(0.0,1.0,0.05), image1=fixed(sitk_s4_2h), image2=fixed(moving_resampled));

interactive(children=(IntSlider(value=975, description='image_z', max=1950), FloatSlider(value=0.5, descriptio…

In [ ]:
rm = sitk.ImageRegistrationMethod()
rm.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
rm.SetMetricSamplingStrategy(rm.RANDOM)
rm.SetMetricSamplingPercentage(0.01)

rm.SetInterpolator(sitk.sitkLinear)

rm.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
rm.SetOptimizerScalesFromPhysicalShift()

rm.SetShrinkFactorsPerLevel(shrinkFactors=[4,2,1])
rm.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
rm.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

rm.SetInitialTransform(initial_transform, inPlace=False)

rm.AddCommand(sitk.sitkStartEvent, regcbs.metric_start_plot)
rm.AddCommand(sitk.sitkEndEvent, regcbs.metric_end_plot)
rm.AddCommand(sitk.sitkMultiResolutionIterationEvent, regcbs.metric_update_multires_iterations)
rm.AddCommand(sitk.sitkIterationEvent, lambda: regcbs.metric_plot_values(rm))
final_transform = rm.Execute(sitk.Cast(sitk_s4_2h, sitk.sitkFloat32), sitk.Cast(sitk_s4_1h, sitk.sitkFloat32))

In [36]:
display(view(s4_1h, s4_2h_bin))

RuntimeError: /work/ITK-source/ITK/Modules/Core/Common/src/itkDataObject.cxx:355:
Requested region is (at least partially) outside the largest possible region.

In [12]:
for name, image in images.items():
    if "2h" in name:
        display(view(image))
        break

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [19]:
for name, image in images.items():
    if "2h" in name:
        display(view(image))
        break

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…